In [1]:
!pip install contractions
!pip install emot
!pip install num2words
!pip install afinn

In [2]:
import pandas as pd
import numpy as np
#Obtaining the datasets
tweets = pd.read_csv('covid19_tweets.csv')

In [3]:
import nltk
from nltk.corpus import gutenberg, stopwords, wordnet
import contractions
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
from bs4 import BeautifulSoup
from num2words import num2words
from emot.emo_unicode import UNICODE_EMOJI #for EMOJIS
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [33]:
def lower_case(text):
  return text.lower()
def rem_lines(text):
    return text.strip().replace('\n', ' ')
def rem_tags(text):
  return BeautifulSoup(text, "html.parser").get_text()
def fix_ct(text):
    return contractions.fix(text)
def rem_stopwords(text):
  stop_words = set(stopwords.words('english'))
  return [word for word in word_tokenize(text) if not word in stop_words]
def rem_punct(text):
    no_punct = [w.translate(str.maketrans('', '', string.punctuation)) for w in word_tokenize(text)]
    return [word for word in no_punct if word.isalpha()]
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
def to_number(text):
    return(re.sub(r"(\d+)", lambda x: num2words(int(x.group(0))), text))
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, EMOTICONS_EMO[emot]+" ".replace("",""))
    return text
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot,UNICODE_EMOJI[emot]+" ".replace("",""))
    return text
def clean_text(text):
  text = text.lower() # convert to lowercase
  text=re.sub(r'[@#]', '', text) #remove tags
  return text

In [5]:
#cleaning tweets
tweets['text']=tweets['text'].astype(str).apply(lower_case)
tweets['text']=tweets['text'].astype(str).apply(rem_lines)
tweets['text']=tweets['text'].astype(str).apply(rem_tags)
tweets['text']=tweets['text'].astype(str).apply(convert_emoticons)
#tweets['text']=tweets['text'].astype(str).apply(convert_emojis)
tweets['text']=tweets['text'].astype(str).apply(fix_ct)
tweets['text']=tweets['text'].astype(str).apply(rem_stopwords)
tweets['text']=tweets['text'].astype(str).apply(rem_punct)
tweets['text']=tweets['text'].astype(str).apply(to_number)
tweets['text']=tweets['text'].astype(str).apply(lemmatize_words)

<ipython-input-4-b21a517f00ba>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [6]:
#sentiment analysis using afinn
from afinn import Afinn
afn = Afinn()
scores = [afn.score(article) for article in tweets['text']]
sentiment = ['positive' if score > 0
                          else 'negative' if score < 0
                              else 'neutral'
                                  for score in scores]
tweets['sentiment']=sentiment
tweets['score']=scores
print(tweets.loc[:,['text','sentiment','score']])

                                                     text sentiment  score
0       ['smelled', 'scent', 'hand', 'sanitizers', 'to...  negative   -7.0
1       ['hey', 'yankees', 'yankeespr', 'mlb', 'would'...  negative   -6.0
2       ['wdunlap', 'realdonaldtrump', 'trump', 'never...  negative   -9.0
3       ['brookbanktv', 'one', 'gift', 'give', 'apprec...  negative   -3.0
4       ['july', 'media', 'bulletin', 'novel', 'corona...  negative   -5.0
...                                                   ...       ...    ...
179103  ['thanks', 'iamohmai', 'nominating', 'wearamas...  negative   -6.0
179104  ['year', 'insanity', 'lol', 'httpsSkeptical', ...  negative   -6.0
179105  ['ctvnews', 'powerful', 'painting', 'juan', 'l...  negative   -6.0
179106  ['students', 'test', 'positive', 'major', 'uni...  negative   -5.0
179107  ['stop', 'see', 'stop', 'sabcnews', 'dailysuns...  negative   -9.0

[179108 rows x 3 columns]


In [7]:
tweets_ascending = tweets.sort_values(by='score', ascending=True)
tweets_descending = tweets.sort_values(by='score', ascending=False)
print(tweets_ascending.loc[:,['text','sentiment','score']])
print(tweets_descending.loc[:,['text','sentiment','score']])

                                                     text sentiment  score
139784  ['imhere', 'httpsSkeptical', 'annoyed', 'undec...  negative  -35.0
65260   ['feel', 'like', 'keeping', 'death', 'watch', ...  negative  -32.0
22760   ['okay', 'snapchat', 'piece', 'fucking', 'shit...  negative  -29.0
47711   ['mywestin', 'oregongovbrown', 'httpsSkeptical...  negative  -28.0
124441  ['facebook', 'httpsSkeptical', 'annoyed', 'und...  negative  -28.0
...                                                   ...       ...    ...
83331   ['breesanna', 'great', 'interview', 'anna', 'p...  positive   12.0
39175   ['good', 'meets', 'good', 'common', 'good', 's...  positive   12.0
131985  ['generalstrike', 'would', 'end', 'covid', 'ca...  positive   12.0
49769   ['wow', 'wow', 'wow', 'look', 'terrific', 'eng...  positive   13.0
53745   ['welcome', 'mat', 'helps', 'comics', 'cartoon...  positive   15.0

[179108 rows x 3 columns]
                                                     text sentiment  scor

# Spanish sentiment analysis

In [36]:
!pip install sentiment-analysis-spanish
!pip install keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 27.6 MB/s eta 0:00:00


In [34]:
#Obtaining the datasets
spanish = pd.read_csv('EleccionesColombia.csv')
print(spanish)

                      tweetID  \
0      ID:1008420713391370240   
1      ID:1008420713009762304   
2      ID:1008420706957320193   
3      ID:1008420706344894464   
4      ID:1008420700787433473   
...                       ...   
29995  ID:1008317884500795392   
29996  ID:1008317883900944384   
29997  ID:1008317879689928705   
29998  ID:1008317872865718273   
29999  ID:1008317856801591296   

                                               tweetText  \
0      @herbinhoyos @petrogustavo @FiscaliaCol El lad...   
1                      @ELTIEMPO @IvanDuque vende patria   
2      @_El_Patriota @PoliciaColombia @petrogustavo l...   
3      @petrogustavo Es verdad! Yo ya lo hice. https:...   
4      La guerra sucia que muestra el desespero de lo...   
...                                                  ...   
29995  El que es sinvergüenza, deja esperando al que ...   
29996  @molemc79 @jarax12 @christian__rev @mejornonon...   
29997  @PalomaValenciaL @IvanDuque Parece una loca!!!...   
299

In [35]:
#cleaning tweets
spanish['tweetText']=spanish['tweetText'].astype(str).apply(clean_text)

0        herbinhoyos petrogustavo fiscaliacol el ladrón...
1                          eltiempo ivanduque vende patria
2        _el_patriota policiacolombia petrogustavo lei ...
3        petrogustavo es verdad! yo ya lo hice. https:/...
4        la guerra sucia que muestra el desespero de lo...
                               ...                        
29995    el que es sinvergüenza, deja esperando al que ...
29996    molemc79 jarax12 christian__rev mejornonono cl...
29997    palomavalencial ivanduque parece una loca!!! n...
29998    nomaslapuntica_ elespectador ivanduque muchos ...
29999    alvarouribevel ivanduque mluciaramirez doctor,...
Name: tweetText, Length: 30000, dtype: object


In [37]:
#sentiment analysis
from sentiment_analysis_spanish import sentiment_analysis
sent_span = sentiment_analysis.SentimentAnalysisSpanish()
scores = [sent_span.sentiment(article) for article in spanish['tweetText']]
sentiment = ['positive' if score > 0.5
                          else 'negative' if score < 0.5
                              else 'neutral'
                                  for score in scores]
spanish['sentiment']=sentiment
spanish['score']=scores
print(spanish.loc[:,['tweetText','sentiment','score']])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


                                               tweetText sentiment  \
0      herbinhoyos petrogustavo fiscaliacol el ladrón...  negative   
1                        eltiempo ivanduque vende patria  negative   
2      _el_patriota policiacolombia petrogustavo lei ...  negative   
3      petrogustavo es verdad! yo ya lo hice. https:/...  negative   
4      la guerra sucia que muestra el desespero de lo...  negative   
...                                                  ...       ...   
29995  el que es sinvergüenza, deja esperando al que ...  negative   
29996  molemc79 jarax12 christian__rev mejornonono cl...  negative   
29997  palomavalencial ivanduque parece una loca!!! n...  negative   
29998  nomaslapuntica_ elespectador ivanduque muchos ...  negative   
29999  alvarouribevel ivanduque mluciaramirez doctor,...  negative   

              score  
0      2.382450e-01  
1      4.978923e-01  
2      7.821248e-02  
3      3.397092e-02  
4      9.762036e-08  
...             ...  
29995

In [39]:
#sorting for easier analysis
spanish_ascending = spanish.sort_values(by='score', ascending=True)
spanish_descending = spanish.sort_values(by='score', ascending=False)
print(spanish_ascending.loc[:,['tweetText','sentiment','score']])
print(spanish_descending.loc[:,['tweetText','sentiment','score']])

                                               tweetText sentiment  \
29647  juanescor apoilonia ivanduque yo no soy petris...  negative   
28153  cecastaneda petrogustavo a_ordonezm 16oscarsan...  negative   
29279  olaznogjunior jessicacediel ivanduque que una ...  negative   
11424  petrogustavo hablan de petro diciendo que es d...  negative   
8283   jjtellezf petrogustavo resulta que no es porqu...  negative   
...                                                  ...       ...   
25497  en el hotel ghl de la ciudad de barranquilla e...  positive   
28001  saludhernandezm cuando petrogustavo fue alcald...  positive   
24286  apoilonia ivanduque el odio y el resentimiento...  positive   
25674  elcolombiano petrogustavo nótese la brecha y e...  positive   
16959  elecciones ivanduque  esperará los resultados ...  positive   

              score  
29647  3.031789e-23  
28153  1.880022e-21  
29279  3.530098e-21  
11424  1.334412e-20  
8283   1.569570e-20  
...             ...  
25497

In [44]:
print(spanish.loc[25497,"tweetText"])

en el hotel ghl de la ciudad de barranquilla el ex gobernador del atlantico y ex ministro de minas carlos rodado noriega, en compra de voto por el candidato ivan duque costo de este 70000 mol pesos,  para refrigerios y transporte en taxi o buses.gustavobolivar petrogustavo https://t.co/w4m1fcr6ov
